# Model 2

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [ ]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_2 = pd.read_csv("data/informatikkurse.csv")
data_model_2 = data_model_2[data_model_2['model_2'] == 1].copy()
data_model_2

## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [ ]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [ ]:
# Anzeige der Spaltennamen von data_model_2
print(data_model_2.columns.tolist())

data_model_2 = data_model_2[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_2.head()

data_model_2 = data_model_2.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_2['titel_kursbesch'] = data_model_2['veranstaltung_titel'] + ' ' + data_model_2['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_2['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

## Stopwords

In [ ]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

# Model

Wir definieren unser Model.

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [ ]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
topic_model_2 = BERTopic(
  embedding_model=embedding_model,
  # min_topic_size=100,
  nr_topics="auto", 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 20,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [20]:
topic_model_2.fit(docs)
topic_model_2.get_topic_info()

Batches: 100%|██████████| 1336/1336 [00:25<00:00, 51.62it/s] 
2026-02-05 15:31:32,493 - BERTopic - Embedding - Completed ✓
2026-02-05 15:31:32,494 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-05 15:32:01,381 - BERTopic - Dimensionality - Completed ✓
2026-02-05 15:32:01,387 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-05 15:32:08,369 - BERTopic - Cluster - Completed ✓
2026-02-05 15:32:08,369 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-02-05 15:32:09,033 - BERTopic - Representation - Completed ✓
2026-02-05 15:32:09,035 - BERTopic - Topic reduction - Reducing number of topics
2026-02-05 15:32:09,047 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-05 15:32:09,667 - BERTopic - Representation - Completed ✓
2026-02-05 15:32:09,671 - BERTopic - Topic reduction - Reduced number of topics from 58 to 30


,Topic,Count,Name,Representation,Representative_Docs
0,-1,30459,-1_und_in_die_der,"[und, in, die, der, the, and, of, von, zu, wer...",[andere Lehrveranstaltung: ASQ - Informatik-Tr...
1,0,6672,0_und_die_der_in,"[und, die, der, in, the, von, werden, seminar,...",[Inf-WissArb: Wissenschaftliches Arbeiten Lern...
2,1,1023,1_learning_machine_deep_and,"[learning, machine, deep, and, the, of, will, ...",[Deep Learning for Beginners (VHB-Kurs) Neural...
3,2,744,2_security_privacy_sicherheit_and,"[security, privacy, sicherheit, and, it, the, ...",[Seminar: Security and Privacy in Mobile Syste...
4,3,255,3_nlp_language_natural_text,"[nlp, language, natural, text, verfahren, proc...",[Visual Computing Praktikum: Deep Learning for...
5,4,232,4_dsg_eiaps_erwartete_teilnehmerzahl,"[dsg, eiaps, erwartete, teilnehmerzahl, vc, gr...",[DSG-EiAPS-B: Einführung in die Informatik - G...
6,5,229,5_graphics_computergraphik_animation_computerg...,"[graphics, computergraphik, animation, compute...","[Systems Programming in C++ (IN0012, IN2106, I..."
7,6,215,6_rechnerarchitektur_hardware_speicher_parallel,"[rechnerarchitektur, hardware, speicher, paral...",[ProInformatik IV: Rechnerarchitektur und Betr...
8,7,184,7_game_games_spiele_und,"[game, games, spiele, und, spieleentwicklung, ...",[Indie-Games: von der Idee zum Markt (IN2353) ...
9,8,166,8_energieinformatik_energy_01_eesys,"[energieinformatik, energy, 01, eesys, block, ...",[Seminar Energieinformatik Bedingt durch den b...


## Topic Visualiszieren, Outlier Reduzieren, Topics mergen

In [21]:
topic_model_2.visualize_topics()
#topic_model_2.visualize_hierarchy() # Hilft extrem zu sehen, welche Themen zusammengehören

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# # Outlier reduzieren
# # BERTopic auf Test-Daten anwenden
# topics, probs = topic_model_2.transform(docs)
# print(topic_model_2.get_topic_freq())

# # # Outlier reduzieren
# topics = topic_model_2.reduce_outliers(docs, topics)
# topic_model_2.update_topics(docs, topics=topics)

In [22]:
topic_model_2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,30459,-1_und_in_die_der,"[und, in, die, der, the, and, of, von, zu, wer...",[andere Lehrveranstaltung: ASQ - Informatik-Tr...
1,0,6672,0_und_die_der_in,"[und, die, der, in, the, von, werden, seminar,...",[Inf-WissArb: Wissenschaftliches Arbeiten Lern...
2,1,1023,1_learning_machine_deep_and,"[learning, machine, deep, and, the, of, will, ...",[Deep Learning for Beginners (VHB-Kurs) Neural...
3,2,744,2_security_privacy_sicherheit_and,"[security, privacy, sicherheit, and, it, the, ...",[Seminar: Security and Privacy in Mobile Syste...
4,3,255,3_nlp_language_natural_text,"[nlp, language, natural, text, verfahren, proc...",[Visual Computing Praktikum: Deep Learning for...
5,4,232,4_dsg_eiaps_erwartete_teilnehmerzahl,"[dsg, eiaps, erwartete, teilnehmerzahl, vc, gr...",[DSG-EiAPS-B: Einführung in die Informatik - G...
6,5,229,5_graphics_computergraphik_animation_computerg...,"[graphics, computergraphik, animation, compute...","[Systems Programming in C++ (IN0012, IN2106, I..."
7,6,215,6_rechnerarchitektur_hardware_speicher_parallel,"[rechnerarchitektur, hardware, speicher, paral...",[ProInformatik IV: Rechnerarchitektur und Betr...
8,7,184,7_game_games_spiele_und,"[game, games, spiele, und, spieleentwicklung, ...",[Indie-Games: von der Idee zum Markt (IN2353) ...
9,8,166,8_energieinformatik_energy_01_eesys,"[energieinformatik, energy, 01, eesys, block, ...",[Seminar Energieinformatik Bedingt durch den b...


## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [23]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
topic_model_2.update_topics(docs, representation_model=rep)
topic_model_2.get_topic_info() 

100%|██████████| 30/30 [05:26<00:00, 10.89s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,30459,-1_Computer Science Education,[Computer Science Education],[andere Lehrveranstaltung: ASQ - Informatik-Tr...
1,0,6672,0_Scientific Work Education,[Scientific Work Education],[Inf-WissArb: Wissenschaftliches Arbeiten Lern...
2,1,1023,1_Deep Learning Course,[Deep Learning Course],[Deep Learning for Beginners (VHB-Kurs) Neural...
3,2,744,2_Security and Privacy,[Security and Privacy],[Seminar: Security and Privacy in Mobile Syste...
4,3,255,3_Deep Learning Text Mining,[Deep Learning Text Mining],[Visual Computing Praktikum: Deep Learning for...
5,4,232,4_Computer Science Introduction,[Computer Science Introduction],[DSG-EiAPS-B: Einführung in die Informatik - G...
6,5,229,5_Modern C++ Programming,[Modern C++ Programming],"[Systems Programming in C++ (IN0012, IN2106, I..."
7,6,215,6_Computer Architecture and Systems,[Computer Architecture and Systems],[ProInformatik IV: Rechnerarchitektur und Betr...
8,7,184,7_Indie Game Development,[Indie Game Development],[Indie-Games: von der Idee zum Markt (IN2353) ...
9,8,166,8_Energy Informatics Seminar,[Energy Informatics Seminar],[Seminar Energieinformatik Bedingt durch den b...


In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")